In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/mwitiderrick/kerasDO/master/HR_comma_sep.csv")

In [3]:
df.head()
df.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'department', 'salary'],
      dtype='object')

In [4]:
df['salary'].unique()

array(['low', 'medium', 'high'], dtype=object)

In [5]:
feats = ['department', 'salary']
df_final = pd.get_dummies(df, columns=feats, drop_first='first')
print(df_final.head())

   satisfaction_level  last_evaluation  ...  salary_low  salary_medium
0                0.38             0.53  ...           1              0
1                0.80             0.86  ...           0              1
2                0.11             0.88  ...           0              1
3                0.72             0.87  ...           1              0
4                0.37             0.52  ...           1              0

[5 rows x 19 columns]


In [6]:
df_final.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'department_RandD', 'department_accounting',
       'department_hr', 'department_management', 'department_marketing',
       'department_product_mng', 'department_sales', 'department_support',
       'department_technical', 'salary_low', 'salary_medium'],
      dtype='object')

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X = df_final.drop(['left'], axis=1).values
Y = df_final['left'].values

In [9]:
print(X.shape)
print(Y.shape)

(14999, 18)
(14999,)


In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2021)

In [11]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [12]:
print(x_train)
print(x_train.shape)
print(y_train.shape)

[[-0.53726216  1.01093682  0.15385957 ... -0.47514589  1.02387011
  -0.86758418]
 [-0.81878398 -1.03320522 -1.46771358 ... -0.47514589  1.02387011
  -0.86758418]
 [-0.69813177 -0.5075687  -0.656927   ...  2.10461677 -0.97668639
  -0.86758418]
 ...
 [ 0.26708593  0.07647189  0.15385957 ... -0.47514589  1.02387011
  -0.86758418]
 [ 1.11165142  1.41976522 -0.656927   ... -0.47514589 -0.97668639
  -0.86758418]
 [ 0.42795555 -0.44916464  0.15385957 ... -0.47514589  1.02387011
  -0.86758418]]
(11999, 18)
(11999,)


In [13]:
import keras
from keras.layers import Dense
from keras.models import Sequential

In [14]:
feed_model = Sequential()

In [15]:
feed_model.add(Dense(10, input_dim=18, activation="relu", kernel_initializer="uniform"))

In [16]:
feed_model.add(Dense(1, activation="sigmoid", kernel_initializer="uniform"))

In [17]:
!pip install keras-visualizer
from keras_visualizer import visualizer

In [18]:
visualizer(feed_model, format='png', view=True)

In [19]:
feed_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                190       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [20]:
feed_model.compile(optimizer= "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [21]:
feed_model.fit(x_train, y_train, batch_size=10, epochs=1, validation_split=0.1)

1080/1080 [==============================] - 6s 4ms/step - loss: 0.4198 - accuracy: 0.8018 - val_loss: 0.3025 - val_accuracy: 0.8308


In [22]:
y_pred = feed_model.predict(x_test)

In [23]:
print(y_pred)

[[0.07546962]
 [0.81111145]
 [0.09996399]
 ...
 [0.02049423]
 [0.1356364 ]
 [0.17871591]]


In [24]:
y_pred = (y_pred>0.5)

In [25]:
y_pred

array([[False],
       [ True],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [26]:
from sklearn.metrics import confusion_matrix

In [27]:
cm = confusion_matrix(y_test, y_pred)

In [28]:
cm

array([[2205,   70],
       [ 414,  311]])

In [29]:
acc = (2178+ 304)/y_test.shape[0]

In [30]:
acc

0.8273333333333334

In [31]:
# Custom model build

In [32]:
import keras
from keras.layers import Dense, Input


In [33]:
input_layer = Input(shape=(18,))
hidden_layer = Dense(10, activation='relu')(input_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)

feed_custom_model = keras.Model(inputs=input_layer, outputs=output_layer)

In [34]:
feed_custom_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 18)]              0         
                                                                 
 dense_2 (Dense)             (None, 10)                190       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [37]:
feed_custom_model.compile(optimizer= "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
feed_custom_model.fit(x_train, y_train, batch_size=10, epochs=1, validation_split=0.1)

1080/1080 [==============================] - 5s 4ms/step - loss: 0.4592 - accuracy: 0.7780 - val_loss: 0.3397 - val_accuracy: 0.8200


In [36]:
# visualizer(feed_custom_model, filename='custom_nn_model', format='png', view=True)

ValueError: ignored